<h3><strong> Klasyczna funkcja ewaluacji vs Model do ewaluacji </strong></h3>

In [1]:
import chess

import sys, os
sys.path.append(os.path.abspath(".."))
from core.transposition_table import ZobristBoard
from training.value_network.data_manager import board_to_tensor
import torch
from training.value_network.model import SmallResNetValue as Model
import numpy as np
from IPython.display import display, SVG

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

model = Model()
model.load_state_dict(torch.load("../models/value_network/CN2_BN2_RLROP.pth", map_location=device))
model.to(device)
model.eval()

def predict_evaluation(board: chess.Board) -> float:
    X_tensor = board_to_tensor(board).to(device)
    with torch.no_grad():
        logits = model(X_tensor).squeeze(0)
    
    evaluation = logits.item()
    return evaluation

Using device: cuda


In [3]:
import time

board = ZobristBoard()

In [4]:
from core.evaluation import evaluate_board

start = time.time()

best_move = None
best_eval = None

for move in board.legal_moves:
    board.push(move)
    eval = evaluate_board(board)
    if best_eval is None or eval > best_eval:
        best_eval = eval
        best_move = move
    board.pop()
    print(f"Ruch: {move}, Ewaluacja: {eval}")

end = time.time()
print(f"Najlepszy ruch z klasycznej ewaluacji: {best_move}, czas: {end - start:.4f} sekund")

Ruch: g1h3, Ewaluacja: 30
Ruch: g1f3, Ewaluacja: 110
Ruch: b1c3, Ewaluacja: 110
Ruch: b1a3, Ewaluacja: 30
Ruch: h2h3, Ewaluacja: 10
Ruch: g2g3, Ewaluacja: -5
Ruch: f2f3, Ewaluacja: -20
Ruch: e2e3, Ewaluacja: 90
Ruch: d2d3, Ewaluacja: 70
Ruch: c2c3, Ewaluacja: 0
Ruch: b2b3, Ewaluacja: -5
Ruch: a2a3, Ewaluacja: 10
Ruch: h2h4, Ewaluacja: 15
Ruch: g2g4, Ewaluacja: 0
Ruch: f2f4, Ewaluacja: -10
Ruch: e2e4, Ewaluacja: 110
Ruch: d2d4, Ewaluacja: 100
Ruch: c2c4, Ewaluacja: 10
Ruch: b2b4, Ewaluacja: 0
Ruch: a2a4, Ewaluacja: 15
Najlepszy ruch z klasycznej ewaluacji: g1f3, czas: 0.0086 sekund


In [5]:
start = time.time()

best_move = None
best_eval = None

for move in board.legal_moves:
    board.push(move)
    eval = predict_evaluation(board)
    if best_eval is None or eval > best_eval:
        best_eval = eval
        best_move = move
    board.pop()
    print(f"Ruch: {move}, Ewaluacja: {eval}")

end = time.time()
print(f"Najlepszy ruch z sieci NN: {best_move}, czas: {end - start:.4f} sekund")

Ruch: g1h3, Ewaluacja: -0.015619615092873573
Ruch: g1f3, Ewaluacja: -0.010317153297364712
Ruch: b1c3, Ewaluacja: -0.010222209617495537
Ruch: b1a3, Ewaluacja: -0.018704058602452278
Ruch: h2h3, Ewaluacja: -0.017809726297855377
Ruch: g2g3, Ewaluacja: -0.02393387071788311
Ruch: f2f3, Ewaluacja: -0.03332940861582756
Ruch: e2e3, Ewaluacja: -0.020017052069306374
Ruch: d2d3, Ewaluacja: -0.027768857777118683
Ruch: c2c3, Ewaluacja: -0.024639077484607697
Ruch: b2b3, Ewaluacja: -0.01999916508793831
Ruch: a2a3, Ewaluacja: -0.02339237742125988
Ruch: h2h4, Ewaluacja: -0.00876122061163187
Ruch: g2g4, Ewaluacja: -0.01345871388912201
Ruch: f2f4, Ewaluacja: -0.0298949982970953
Ruch: e2e4, Ewaluacja: -0.008586183190345764
Ruch: d2d4, Ewaluacja: -0.013552109710872173
Ruch: c2c4, Ewaluacja: -0.008831293322145939
Ruch: b2b4, Ewaluacja: -0.022525586187839508
Ruch: a2a4, Ewaluacja: -0.023028166964650154
Najlepszy ruch z sieci NN: e2e4, czas: 0.2171 sekund


<hr>

<h3><strong>Minimax z klasyczną funkcją ewaluacji vs Minimax z modelem do ewaluacji </strong></h3>

In [6]:
def minimax_classic(position, depth, alpha, beta, maximizingPlayer):
    if depth == 0 or position.is_game_over():
        return evaluate_board(position), None
    
    best_move = None
    if maximizingPlayer:
        maxEval = float('-inf')
        for move in position.legal_moves:
            position.push(move)
            evaluation, _ = minimax_classic(position, depth-1, alpha, beta, False)
            position.pop()
            if evaluation > maxEval:
                maxEval = evaluation
                best_move = move
            alpha = max(alpha, evaluation)
            if beta <= alpha:
                break
        return maxEval, best_move
    else:
        minEval = float('inf')
        for move in position.legal_moves:
            position.push(move)
            evaluation, _ = minimax_classic(position, depth-1, alpha, beta, True)
            position.pop()
            if evaluation < minEval:
                minEval = evaluation
                best_move = move
            beta = min(beta, evaluation)
            if beta <= alpha:
                break
        return minEval, best_move

In [7]:
def minimax_NN(position, depth, alpha, beta, maximizingPlayer):
    if depth == 0 or position.is_game_over():
        return predict_evaluation(position), None
    
    best_move = None
    if maximizingPlayer:
        maxEval = float('-inf')
        for move in position.legal_moves:
            position.push(move)
            evaluation, _ = minimax_NN(position, depth-1, alpha, beta, False)
            position.pop()
            if evaluation > maxEval:
                maxEval = evaluation
                best_move = move
            alpha = max(alpha, evaluation)
            if beta <= alpha:
                break
        return maxEval, best_move
    else:
        minEval = float('inf')
        for move in position.legal_moves:
            position.push(move)
            evaluation, _ = minimax_NN(position, depth-1, alpha, beta, True)
            position.pop()
            if evaluation < minEval:
                minEval = evaluation
                best_move = move
            beta = min(beta, evaluation)
            if beta <= alpha:
                break
        return minEval, best_move

In [8]:
start = time.time()

eval, best_move = minimax_classic(board, 5, float('-inf'), float('inf'), True)

end = time.time()

print(f"Najlepszy ruch z klasycznego minimax'a: {best_move} z ewaluacją: {eval}, czas: {end - start:.4f} sekund")

start = time.time()

eval, best_move = minimax_NN(board, 5, float('-inf'), float('inf'), True)

end = time.time()

print(f"Najlepszy ruch z NN minimax'a: {best_move} z ewaluacją: {eval}, czas: {end - start:.4f} sekund")


Najlepszy ruch z klasycznego minimax'a: e2e4 z ewaluacją: 135, czas: 16.4480 sekund
Najlepszy ruch z NN minimax'a: e2e3 z ewaluacją: 0.010989324189722538, czas: 44.2482 sekund


In [9]:
start = time.time()

eval, best_move = minimax_classic(board, 6, float('-inf'), float('inf'), True)

end = time.time()

print(f"Najlepszy ruch z klasycznego minimax'a: {best_move} z ewaluacją: {eval}, czas: {end - start:.4f} sekund")

start = time.time()

eval, best_move = minimax_NN(board, 6, float('-inf'), float('inf'), True)

end = time.time()

print(f"Najlepszy ruch z NN minimax'a: {best_move} z ewaluacją: {eval}, czas: {end - start:.4f} sekund")

Najlepszy ruch z klasycznego minimax'a: d2d4 z ewaluacją: -50, czas: 185.2753 sekund
Najlepszy ruch z NN minimax'a: b1c3 z ewaluacją: 0.056217387318611145, czas: 530.4276 sekund
